<a href="https://colab.research.google.com/github/dohyun-kim92/TIL/blob/master/Ngram%ED%9B%84_NBC%2C_polarityScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이 파일은 다음과 같은걸 했습니다.
- json 파일 불러오기 ( 아주 간단하게 pd.read_json )  
- 행오름차순 처리 axis = 0  ( axis = 1 은 열 오름차순 처리 )  
- 행(열) 삭제  
- 행(열) 순서 변경  
- NaN값 제거  
- 리스트 맨앞 '[', 맨뒤']' 제거  (join, lambda, map 사용)  
- 파이프라인에서 하나 지정해서 뽑아서 쓰기 (named_steps['clf'])  
clf = text_clf.named_steps['clf']   
  ( 전처리 끝~ NBC 시작 ) 
  
- NBC

In [ ]:
import requests
import csv
import math
import json
from bs4 import BeautifulSoup
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB



In [ ]:
# ngram 데이터 받아오기
df = pd.read_json(r'C:\Users\dhsmi\Downloads\final_ngram.json')
df

,date,ngram
0,2005-05-12,"[부진/NNG;글로벌/NNG;경기/NNG;둔화/NNG;우려/NNG, 소비/NNG;설..."
1,2005-06-09,"[경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG, 국제/NNG;금융..."
2,2005-06-23,"[대출/NNG;촉진/NNG, 걸림돌/NNG, 개선/NNG]"
3,2005-07-07,"[자산/NNG;가격/NNG;상승/NNG;물가/NNG;상승/NNG, 물가/NNG;안정..."
4,2005-08-11,"[부동산/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG, 국채/NNG;수..."
...,...,...
305067,2020-07-15,"[실망/NNG, 중단/NNG]"
305068,2020-07-15,"[경기/NNG;침체/NNG, 완화/NNG]"
305069,2020-07-15,"[코픽스/NNG;떨어/VV, 돈/NNG;확보/NNG, 하락/NNG, 내림/NNG]"
305070,2020-07-15,"[저축성/NNG;예금/NNG;늘/VV, 경기/NNG;악화/NNG, 통화/NNG;급증..."


In [ ]:
# 회사채 데이터 받아오기
cb = pd.read_json(r'C:\Users\dhsmi\Downloads\labeled_cb_rate.json')
cb

,date,rate,ud
0,2005-04-01,4.33,-1
1,2005-04-02,4.33,-1
2,2005-04-03,4.33,-1
3,2005-04-04,4.35,-1
4,2005-04-05,4.35,-1
...,...,...,...
5538,2020-05-30,2.15,1
5539,2020-05-31,2.15,1
5540,2020-06-01,2.16,1
5541,2020-06-02,2.18,1


In [ ]:
# ngram & 회사체 합체 및 행오름차순 처리 axis = 0  ( axis = 1 은 열 오름차순 처리 )
cb_mg = df.merge(cb , on = 'date', how = 'left')
print(type(cb_mg))

cb_mg = cb_mg.sort_values(by=['date']) #axis = 0 이면 행을 기준으로, axis = 1이면 열을 기준으로 오름차순 
type(cb_mg['ngram'][0])

# NaN값 제거 -> 인덱스도 지워지니까 인덱스 reset 
cb_mg = cb_mg.dropna(subset = ['ngram'])
cb_mg = cb_mg.dropna(subset = ['rate'])
cb_mg


# 처음에 저장할 때 콤마로 조인하면 ngram의 [] 대괄호 지워짐 ..
# 여기서 지우려면 람다펑션 replace 쓰면..


<class 'pandas.core.frame.DataFrame'>


,date,ngram,rate,ud
4001,2005-05-01,"[수입/NNG;증대/NNG, 실적/NNG;높/VV, 하락/NNG, 위험/NNG, 자...",4.18,-1.0
4000,2005-05-01,"[기업/NNG;수익성/NNG;악화/NNG;우려/NNG, 경제/NNG;지표/NNG;성...",4.18,-1.0
3999,2005-05-01,"[인플레이션/NNG;압력/NNG;억제/NNG, 금리/NNG;인상/NNG;긴축/NNG...",4.18,-1.0
232706,2005-05-01,"[대출/NNG;연체율/NNG;오르/VV, 대출/NNG;연체율/NNG;높/VA, 대출...",4.18,-1.0
232705,2005-05-01,"[무수익여신/NNG;증가/NNG, 수익성/NNG;악화/NNG, 악화/NNG;조짐/N...",4.18,-1.0
...,...,...,...,...
292866,2020-06-03,"[대부업/NNG;고금리/NNG, 최대/NNG]",2.19,1.0
292867,2020-06-03,"[매출/NNG;감소/NNG, 금리/NNG;낮/VA, 타격/NNG, 최저/NNG, 최...",2.19,1.0
292868,2020-06-03,"[안정/NNG, 유예/NNG, 최대/NNG]",2.19,1.0
231808,2020-06-03,"[금융채/NNG;발행/NNG;급증/NNG, 금리/NNG;비교/NNG;낮/VA]",2.19,1.0


In [ ]:
# 리스트 맨앞 '[', 맨뒤']' 제거  (join, lambda, map 사용)
cb_mg['ngram'] = list(map(lambda i : ','.join(i), cb_mg['ngram']))
cb_mg



,date,ngram,rate,ud
4001,2005-05-01,"수입/NNG;증대/NNG,실적/NNG;높/VV,하락/NNG,위험/NNG,자신/NNG",4.18,-1.0
4000,2005-05-01,"기업/NNG;수익성/NNG;악화/NNG;우려/NNG,경제/NNG;지표/NNG;성장/...",4.18,-1.0
3999,2005-05-01,"인플레이션/NNG;압력/NNG;억제/NNG,금리/NNG;인상/NNG;긴축/NNG,지...",4.18,-1.0
232706,2005-05-01,"대출/NNG;연체율/NNG;오르/VV,대출/NNG;연체율/NNG;높/VA,대출심사/...",4.18,-1.0
232705,2005-05-01,"무수익여신/NNG;증가/NNG,수익성/NNG;악화/NNG,악화/NNG;조짐/NNG,...",4.18,-1.0
...,...,...,...,...
292866,2020-06-03,"대부업/NNG;고금리/NNG,최대/NNG",2.19,1.0
292867,2020-06-03,"매출/NNG;감소/NNG,금리/NNG;낮/VA,타격/NNG,최저/NNG,최대/NNG",2.19,1.0
292868,2020-06-03,"안정/NNG,유예/NNG,최대/NNG",2.19,1.0
231808,2020-06-03,"금융채/NNG;발행/NNG;급증/NNG,금리/NNG;비교/NNG;낮/VA",2.19,1.0


In [ ]:
cb_mg[cb_mg['ud']==-1]

,date,ngram,rate,ud
4001,2005-05-01,"수입/NNG;증대/NNG,실적/NNG;높/VV,하락/NNG,위험/NNG,자신/NNG",4.18,-1.0
4000,2005-05-01,"기업/NNG;수익성/NNG;악화/NNG;우려/NNG,경제/NNG;지표/NNG;성장/...",4.18,-1.0
3999,2005-05-01,"인플레이션/NNG;압력/NNG;억제/NNG,금리/NNG;인상/NNG;긴축/NNG,지...",4.18,-1.0
232706,2005-05-01,"대출/NNG;연체율/NNG;오르/VV,대출/NNG;연체율/NNG;높/VA,대출심사/...",4.18,-1.0
232705,2005-05-01,"무수익여신/NNG;증가/NNG,수익성/NNG;악화/NNG,악화/NNG;조짐/NNG,...",4.18,-1.0
...,...,...,...,...
231094,2020-05-04,"우려/NNG;금리/NNG;하락/NNG,기대/NNG;금리/NNG;상승/NNG,물량/N...",2.20,-1.0
231095,2020-05-04,"금리/NNG;하락/NNG;이자비용/NNG;감소/NNG,차입/NNG;금리/NNG;하락...",2.20,-1.0
231096,2020-05-04,"기업/NNG;실적/NNG;개선/NNG,달러원/NNG;넘/VV;오르/VV,gdp/NN...",2.20,-1.0
231097,2020-05-04,"저성장/NNG;국면/NNG;벗어나/VV,글로벌/NNG;교역/NNG;둔화/NNG,이익...",2.20,-1.0


In [ ]:
# 각 class 수 아래서 multiNB 수 잘나왔나 확인하기 위함 
test_tmp = []
for i in range(len(cb_mg)):
    test_tmp.append(cb_mg['ud'].iloc[i])
test_tmp

[-1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,

In [ ]:
# 각 class 수 아래서 multiNB 수 잘나왔나 확인하기 위함 
cb_mg.iloc[0]

date                                2005-05-01 00:00:00
ngram    수입/NNG;증대/NNG,실적/NNG;높/VV,하락/NNG,위험/NNG,자신/NNG
rate                                               4.18
ud                                                   -1
Name: 4001, dtype: object

In [ ]:
# 5월전 행 삭제 , ngram = NaN 삭제

cb_mg = cb_mg[cb_mg.date >= '2005-05-01'] 
# cb_mg = cb_mg[cb_mg.date != '[]']
# cb_mg['ngram'][30]



In [ ]:
# 컬럼 순서 변경 

cb_mg = cb_mg[['date', 'rate', 'ngram', 'ud']]
cb_mg

# type(cb_mg["ngram"][0])

len(cb_mg["ngram"][0])

3335

## NBC
-1은 dovish 1은 hokish // ngram(x)랑 y(ud) 사용 ...  date랑 rate 는 ngram, ud 뽑는데 쓴거라 이제 필요없음 ~

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import numpy as np

# train_test_split? 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cb_mg['ngram'], cb_mg['ud'], train_size= 0.9, random_state = 0)

def com_split(x):
    return x.split(",")

text_clf = Pipeline([   ('cv',CountVectorizer(tokenizer= com_split)), # split 뒤에 (x) 써줄필요 없음 
#                         ('tfidfV', TfidfVectorizer()), -> 논문에 따르면 tfidf 써줄 필요 없음 
                        ('clf', MultinomialNB())
])                                                      # 아래꺼안쓰는 이유는 우리는 ngram 해놨으니까~ tfidfvectorizer로씀 
                                                        # tfidftransformer는 countvect에서 넘어오는거라 안됨 


# ud[1].class_count_ , cb_mg['ud'].feature_count_
text_clf.fit(X_train, y_train) # 처음에 에러떴던 이유는 X_trian에서 대괄호('[]')를 제거하지 않았기때문 


clf = text_clf.named_steps['clf']
print(clf.class_count_, "\n", clf.feature_count_) 
print("{} : {}".format('clf.classes_',clf.classes_))

print(sum(clf.feature_count_[0]))
print(sum(clf.feature_count_[1]))
print(sum(clf.feature_count_[2]))

# predicted = text_clf.predict(X_test)
# np.mean(predicted == y_test)
# text_clf
clf.feature_count_.shape


[114171.  52605.  95669.] 
 [[ 1.  2.  3. ... 28.  5.  5.]
 [ 2.  2.  2. ... 12.  5.  1.]
 [ 1.  5.  2. ... 12.  7.  3.]]
clf.classes_ : [-1.  0.  1.]
1247237.0
580003.0
1041079.0


(3, 65027)

# 위에까진 완료

In [ ]:
# Unique n_gram 수 확인 
import itertools
temp = list(map(lambda i : i.split(','), X_train))
len(set(list(itertools.chain(*temp))))

65027

In [ ]:
# polarity score 계산 
p_score = []  

for i in range(len(clf.feature_log_prob_[0])):
    p_score.append(
                  (clf.feature_log_prob_[0][i] / sum(clf.feature_log_prob_[0])) 
        /
        (clf.feature_log_prob_[2][i] / sum(clf.feature_log_prob_[2]))
                  )
"분자"  
"분모"
    
    
p_score


# 이거 막으려면 log 써라  분모가 0 이어서... warning 뜸 

KeyboardInterrupt: 

In [ ]:
sum(clf.class_count_)

262445.0

In [ ]:
np.sum(clf.feature_count_[0])


1247237.0

In [ ]:
clf.classes_

array([-1.,  0.,  1.])

In [ ]:
print(predicted[:200])
len(predicted)
cb_mg["ud"].class_count_

NameError: name 'predicted' is not defined

In [ ]:
# d / h 의 조건부 확률 비율로 polarity score 산출 

In [ ]:
test = "나는 오늘 시험에서 빵점을 맞았어요, 하지만 나는 바보는 아니에요, 그저 공부를 안했을 뿐이랍니다"
# test.split(",")

def com_split(x):
    return x.split(",")

com_split(test)

In [ ]:
CountVectorizer(tokenizer= com_split(cb_mg['ngram']))


In [ ]:
k = 30
pol_list = []
for i in range(k):
    ~~
    pol_list.append()
    
# feature_count_ , class_count_ 
# cb_mg['ngram'].feature_count_  -> cb_mg 자리에 x_train ??
# ud[1].class_count_ , cb_mg['ud'].feature_count_

In [ ]:
"분자"  = math.exp(math.log())
"분모"